In [2]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import pandas as pd
import numpy as np

In [3]:
df= pd.read_csv("C:/Users/yasin/Desktop/birlesik_yorumlar.csv")

In [4]:
df

,0
0,"“The fries were terrific also, hot crisp...”"
1,“I love the food and our server Maria!”
2,“The filet mignon was impeccable and the musse...
3,“My seafood cocktail had wonderful large lump ...
4,“penne al pomodoro and bucatini cacio e pepe w...
...,...
17393,"The room was good, comfortable and aesthetic \..."
17394,good hotel
17395,good experience for me about hotel \nvery good...
17396,well done


In [5]:
#Varsayalım ki DataFrame'iniz df ve yorumlar '0' sütununda
text = ' '.join(df['0'].astype(str).tolist())  # Tüm yorumları tek bir metin haline getir
sentences = sent_tokenize(text)  # Metni cümlelere ayır


In [6]:
# Lemmatizer ve Stemmer'ı başlat
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [7]:
# Stopwords listesini almak
stop_words = set(stopwords.words('english'))

In [8]:
# Kelimeleri tokenleştirip, lemmatize etme ve stemleme
def preprocess_sentence(sentence):
    tokens = word_tokenize(sentence)  # Cümleyi kelimelere ayır
    # Sadece harf olan kelimeleri al ve stopword'leri çıkar
    filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]  # Lemmatize etme
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]  # Stemleme
    
    return lemmatized_tokens, stemmed_tokens

In [9]:
# Her cümleyi tokenleştir, lemmatize et ve stemle
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []

In [10]:
for sentence in sentences:
    lemmatized_tokens, stemmed_tokens = preprocess_sentence(sentence)
    tokenized_corpus_lemmatized.append(lemmatized_tokens)
    tokenized_corpus_stemmed.append(stemmed_tokens)

In [11]:
# İlk 5 cümleyi yazdıralım
for i in range(5):
 print(f"Cümle {i+1} - Lemmatized: {tokenized_corpus_lemmatized[i]}")
 print(f"Cümle {i+1} - Stemmed: {tokenized_corpus_stemmed[i]}")
 print("\n")

Cümle 1 - Lemmatized: ['fry', 'terrific', 'also', 'hot', 'crisp', 'love', 'food', 'server', 'maria', 'filet', 'mignon', 'impeccable', 'mussel', 'steak', 'tartare', 'also', 'v', 'seafood', 'cocktail', 'wonderful', 'large', 'lump', 'crabmeat', 'delicious', 'lobster', 'penne', 'al', 'pomodoro', 'bucatini', 'cacio', 'e', 'pepe', 'highly', 'recommend', 'onion', 'soup', 'mussel', 'french', 'great', 'food', 'good', 'price', 'best', 'cocktail', 'worth', 'travelling', 'mile', 'incredible', 'italian', 'perfect', 'pizza', 'top', 'splendid', 'pizza', 'excellent', 'french', 'onion', 'soup', 'food', 'great', 'especially', 'mussel', 'good', 'fresh']
Cümle 1 - Stemmed: ['fri', 'terrif', 'also', 'hot', 'crisp', 'love', 'food', 'server', 'maria', 'filet', 'mignon', 'impecc', 'mussel', 'steak', 'tartar', 'also', 'v', 'seafood', 'cocktail', 'wonder', 'larg', 'lump', 'crabmeat', 'delici', 'lobster', 'penn', 'al', 'pomodoro', 'bucatini', 'cacio', 'e', 'pepe', 'highli', 'recommend', 'onion', 'soup', 'mussel'

In [12]:
import gensim
from gensim.models import Word2Vec

In [17]:
# Kelimeleri tokenleştirip, lemmatize etme ve stemleme
def preprocess_sentence(sentence):
 tokens = word_tokenize(sentence) # Cümleyi kelimelere ayır
# Sadece harf olan kelimeleri al ve stopword'leri çıkar
 filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
 lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens] # Lemmatize etme
 stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens] #Stemleme
 return lemmatized_tokens, stemmed_tokens

In [18]:
# Her cümleyi tokenleştir, lemmatize et ve stemle
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []

In [20]:
for sentence in sentences:
 lemmatized_tokens, stemmed_tokens = preprocess_sentence(sentence)
 tokenized_corpus_lemmatized.append(lemmatized_tokens)
 tokenized_corpus_stemmed.append(stemmed_tokens)

In [21]:
# Word2Vec modeli eğitmek için parametreler
parameters = [
   {'model_type': 'cbow', 'window': 2, 'vector_size': 100},
   {'model_type': 'skipgram', 'window': 2, 'vector_size': 100},
   {'model_type': 'cbow', 'window': 4, 'vector_size': 100},
   {'model_type': 'skipgram', 'window': 4, 'vector_size': 100},
   {'model_type': 'cbow', 'window': 2, 'vector_size': 300},
   {'model_type': 'skipgram', 'window': 2, 'vector_size': 300},
   {'model_type': 'cbow', 'window': 4, 'vector_size': 300},
   {'model_type': 'skipgram', 'window': 4, 'vector_size': 300}
]


In [27]:
# Fonksiyon ile Word2Vec modeli eğitme ve kaydetme
def train_and_save_model(corpus, params, model_name):
    model = Word2Vec(corpus, vector_size=params['vector_size'], window=params['window'], min_count=1, sg=1 if params['model_type'] == 'skipgram' else 0)
    model.save(f"{model_name}_{params['model_type']}_window{params['window']}_dim{params['vector_size']}.model")
    print(f"{model_name}_{params['model_type']}_window{params['window']}_dim{params['vector_size']} model saved!")

In [28]:
# Lemmatize edilmiş corpus ile modelleri eğitme ve kaydetme
for param in parameters:
    train_and_save_model(tokenized_corpus_lemmatized, param, "lemmatized_model")

# Stemlenmiş corpus ile modelleri eğitme ve kaydetme
for param in parameters:
    train_and_save_model(tokenized_corpus_stemmed, param, "stemmed_model")

lemmatized_model_cbow_window2_dim100 model saved!
lemmatized_model_skipgram_window2_dim100 model saved!
lemmatized_model_cbow_window4_dim100 model saved!
lemmatized_model_skipgram_window4_dim100 model saved!
lemmatized_model_cbow_window2_dim300 model saved!
lemmatized_model_skipgram_window2_dim300 model saved!
lemmatized_model_cbow_window4_dim300 model saved!
lemmatized_model_skipgram_window4_dim300 model saved!
stemmed_model_cbow_window2_dim100 model saved!
stemmed_model_skipgram_window2_dim100 model saved!
stemmed_model_cbow_window4_dim100 model saved!
stemmed_model_skipgram_window4_dim100 model saved!
stemmed_model_cbow_window2_dim300 model saved!
stemmed_model_skipgram_window2_dim300 model saved!
stemmed_model_cbow_window4_dim300 model saved!
stemmed_model_skipgram_window4_dim300 model saved!


In [39]:
# Model dosyalarını yüklemek
model_1 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/lemmatized_model_cbow_window2_dim100.model")
model_2 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/lemmatized_model_cbow_window4_dim100.model")
model_3 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/lemmatized_model_cbow_window2_dim300.model")
model_4 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/lemmatized_model_cbow_window4_dim300.model")
model_5 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/lemmatized_model_skipgram_window2_dim100.model")
model_6 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/lemmatized_model_skipgram_window2_dim300.model")
model_7 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/lemmatized_model_skipgram_window4_dim100.model")
model_8 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/lemmatized_model_skipgram_window4_dim300.model")
model_9 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/stemmed_model_cbow_window2_dim100.model")
model_10 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/stemmed_model_cbow_window2_dim300.model")
model_11= Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/stemmed_model_cbow_window4_dim100.model")
model_12= Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/stemmed_model_cbow_window4_dim300.model")
model_13= Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/stemmed_model_skipgram_window2_dim100.model")
model_14= Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/stemmed_model_skipgram_window2_dim300.model")
model_15 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/stemmed_model_skipgram_window4_dim100.model")
model_16 = Word2Vec.load("C:/Users/yasin/Desktop/2.hafta/stemmed_model_skipgram_window4_dim300.model")

In [42]:
# 'python' kelimesi ile en benzer 3 kelimeyi ve skorlarını yazdırmak
def print_similar_words(model, model_name):
    similarity = model.wv.most_similar('soup', topn=3)
    print(f"\n{model_name} Modeli - 'soup' ile En Benzer 3 Kelime:")
    for word, score in similarity:
        print(f"Kelime: {word}, Benzerlik Skoru: {score}")

In [44]:
print_similar_words(model_1 , "lemmatized_model_cbow_window2_dim100.model")
print_similar_words(model_2 , "lemmatized_model_cbow_window4_dim100.model")
print_similar_words(model_3 , "lemmatized_model_cbow_window2_dim300.model")
print_similar_words(model_4, "lemmatized_model_cbow_window4_dim300.model")
print_similar_words(model_5, "lemmatized_model_skipgram_window2_dim100.model")
print_similar_words(model_6, "lemmatized_model_skipgram_window2_dim300.model")
print_similar_words(model_7 , "lemmatized_model_skipgram_window4_dim100.model")
print_similar_words(model_8, "lemmatized_model_skipgram_window4_dim300.model")
print_similar_words(model_9 , "stemmed_model_cbow_window2_dim100.model")
print_similar_words(model_10, "stemmed_model_cbow_window2_dim300.model")
print_similar_words(model_11, "stemmed_model_cbow_window4_dim100.model")
print_similar_words(model_12, "stemmed_model_cbow_window4_dim300.model")
print_similar_words(model_13, "stemmed_model_skipgram_window2_dim100.model")
print_similar_words(model_14, "stemmed_model_skipgram_window2_dim300.model")
print_similar_words(model_15, "stemmed_model_skipgram_window4_dim100.model")
print_similar_words(model_16, "stemmed_model_skipgram_window4_dim300.model")


lemmatized_model_cbow_window2_dim100.model Modeli - 'soup' ile En Benzer 3 Kelime:
Kelime: chicken, Benzerlik Skoru: 0.9977431893348694
Kelime: fried, Benzerlik Skoru: 0.9969526529312134
Kelime: pasta, Benzerlik Skoru: 0.9965759515762329

lemmatized_model_cbow_window4_dim100.model Modeli - 'soup' ile En Benzer 3 Kelime:
Kelime: onion, Benzerlik Skoru: 0.9983174204826355
Kelime: egg, Benzerlik Skoru: 0.9982950687408447
Kelime: dish, Benzerlik Skoru: 0.9973147511482239

lemmatized_model_cbow_window2_dim300.model Modeli - 'soup' ile En Benzer 3 Kelime:
Kelime: authentic, Benzerlik Skoru: 0.9981626272201538
Kelime: thin, Benzerlik Skoru: 0.9981129765510559
Kelime: nyc, Benzerlik Skoru: 0.9980076551437378

lemmatized_model_cbow_window4_dim300.model Modeli - 'soup' ile En Benzer 3 Kelime:
Kelime: onion, Benzerlik Skoru: 0.9994454383850098
Kelime: nyc, Benzerlik Skoru: 0.9987310767173767
Kelime: lobster, Benzerlik Skoru: 0.9984368085861206

lemmatized_model_skipgram_window2_dim100.model Mode